In [11]:
import pandas as pd
import numpy as np

In [29]:
def datFormat(file_name):
    # Adding Format to .dat file and saving to .csv
    
    #Reading .dat file
    datFile = pd.read_fwf(file_name, header=None, index_col=False,
                          colspecs=[(0, 4), (4, 5), (5, 50), (50, 60), (60, 90),
                                    (90, 113), (113, 117), (117, 132), (132, 138), (138, 148),
                                    (148, 156), (156, 166), (166, 216), (216, 224), (224, 274),
                                    (274, 284), (284, 293), (293, 298), (298, 299), (299, 306),
                                    (307, 317), (316, 324), (324, 332), (332, 340), (340, 341),
                                    (341, 349), (349, 389), (389, 400), (400, 405), (405, 450),
                                    (450, 465), (465, 469)],
                          names=["DIST-YEAR", "DIST-QTR", "SUPPLIER-NAME", "SONG-CODE", "SONG-TITLE",
                                 "SONG-WRITER", "INCOME-TYPE", "EARNINGS-AMOUNT", "SHARE-PERCENT", "SOCIETY",
                                 "SUPPLIER-NO", "AV-WORK-NO", "AV-WORK-TITLE", "SERIES-NO", "SERIES-TITLE",
                                 "SOURCE", "PERF-COUNT", "PERF-QTR", "USAGE", "DURATION",
                                 "IPI NAME NO", "DIST-DATE", "DIST-PERIOD-START", "DIST-PERIOD-END", "SPECIAL-DIST",
                                 "PERF-DATE", "PERFORMERS", "ISWC", "STATION", "VENUE",
                                 "CITY", "PERF-TYPE"])
    dataFrame = pd.DataFrame(data=datFile)

    # Formatting .dat file
    dataFrame = dataFrame.fillna("")
    dataFrame["DIST-PERIOD-START"] = pd.to_datetime(dataFrame["DIST-PERIOD-START"], format="%Y%m%d")
    dataFrame["DIST-PERIOD-END"] = pd.to_datetime(dataFrame["DIST-PERIOD-END"], format="%Y%m%d")
    dataFrame["EARNINGS-AMOUNT"] = dataFrame["EARNINGS-AMOUNT"].astype(float) / 10000
    dataFrame["DURATION"] = "00000000" + dataFrame["DURATION"].astype(str).str[:-2]
    dataFrame["DURATION"] = dataFrame["DURATION"].str[-6:-4] + ":" + \
                            dataFrame["DURATION"].str[-4:-2] + ":" + dataFrame["DURATION"].str[-2:]
    dataFrame = dataFrame.replace({"DURATION": {"00:00:00": np.nan}})
    dataFrame["SOURCE"] = dataFrame["SOURCE"] + dataFrame["STATION"]
    dataFrame["SHARE-PERCENT"] = dataFrame["SHARE-PERCENT"].astype(float) / 1000
    dataFrame = dataFrame.replace({"SOURCE": {"ISPOT": "ISPOTIFY"}})
    dataFrame["DIST."] = dataFrame["DIST-QTR"].astype(str) + "Q" + dataFrame["DIST-YEAR"].astype(str).str[-2:]
    dataFrame["PERF-COUNT"] = pd.to_numeric(dataFrame["PERF-COUNT"])
    dataFrame = dataFrame.rename(columns={"ISWC": "ISWC-DAT"})

    # Rearrange columns
    dataFrame = dataFrame[["DIST.", "DIST-DATE", "DIST-PERIOD-START", "DIST-PERIOD-END", "SPECIAL-DIST",
                           "SUPPLIER-NAME", "SUPPLIER-NO", "IPI NAME NO",
                           "SERIES-NO", "SERIES-TITLE", "AV-WORK-NO", "AV-WORK-TITLE",
                           "SONG-TITLE", "SONG-WRITER", "SONG-CODE",
                           "ISWC-DAT", "SHARE-PERCENT", "USAGE", "PERF-COUNT", "SOCIETY", "INCOME-TYPE",
                           "SOURCE", "DURATION", "PERF-DATE", "PERFORMERS", "VENUE", "CITY", "PERF-TYPE",
                           "EARNINGS-AMOUNT"]]
    # Saving .dat into .csv
    dataFrame.to_csv("./B01_SOCAN_2012toJun2022_Dat.csv", encoding="utf-8", index=False)
    return

In [30]:
def cleanDataFrames():
    df1 = pd.DataFrame(pd.read_csv("./B01_SOCAN_2012toJun2022_Dat.csv", index_col=False))
    df2 = pd.DataFrame(pd.read_csv("./Bedtracks for Diana/B01_SOCAN_2012toJun2022.csv", index_col=False))
    
    df1 = df1.fillna("")
    
    # DataFrame #2
    df2 = df2.fillna("")
    df2 = df2.rename(columns={" Work Number": "Work Number"})
    df2["Work Number"] = pd.to_numeric(df2["Work Number"])
    df2["Distribution Date"] = pd.to_datetime(df2["Distribution Date"])
    df2["Dist. Period Start"] = pd.to_datetime(df2["Dist. Period Start"])
    df2["Dist. Period End"] = pd.to_datetime(df2["Dist. Period End"])

    return df1, df2

In [31]:
datFormat("./Bedtracks for Diana/B01_SOCAN_2012toJun2022.dat")

In [32]:
df_dat, df_csv = cleanDataFrames()

# Merging both files into one with entries side by side
df_comparison = pd.concat([df_csv, df_dat], axis=1)
df_comparison.to_csv("./BOTH_B01_SOCAN_2012toJun2022.csv", encoding="utf-8", index=False)

/var/folders/xw/90drjjz96lnbkt1qdfd7vqp00000gn/T/ipykernel_16514/3646952792.py:2: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.DataFrame(pd.read_csv("./B01_SOCAN_2012toJun2022_Dat.csv", index_col=False))
/var/folders/xw/90drjjz96lnbkt1qdfd7vqp00000gn/T/ipykernel_16514/3646952792.py:3: DtypeWarning: Columns (24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.DataFrame(pd.read_csv("./Bedtracks for Diana/B01_SOCAN_2012toJun2022.csv", index_col=False))


In [48]:
# Adding column to note if different information in the files
df_comparison["Difference"] = np.nan
df_comparison.loc[df_comparison["Dist."] != df_comparison["DIST."], "Difference"] = "Dist.&DIST."
df_comparison.loc[df_comparison["Member Name"] != df_comparison["SUPPLIER-NAME"], "Difference"] = "Member Name&SUPPLIER-NAME"
df_comparison.loc[df_comparison["AV Title"] != df_comparison["SERIES-TITLE"], "Difference"] = "AV Title&SERIES-TITLE"
df_comparison.loc[df_comparison["Episode"] != df_comparison["AV-WORK-TITLE"], "Difference"] = "Episode&AV-WORK-TITLE"
df_comparison.loc[df_comparison["Work Number"] != df_comparison["SONG-CODE"], "Difference"] = "Work Number&SONG-CODE"
df_comparison.loc[df_comparison["ISWC"] != df_comparison["ISWC-DAT"], "Difference"] = "ISWC&ISWC-DAT"
df_comparison.loc[df_comparison["Use"] != df_comparison["USAGE"], "Difference"] = "Use&USAGE"
df_comparison.loc[df_comparison["# of Plays"] != df_comparison["PERF-COUNT"], "Difference"] = "Plays&PERF-COUNT"
df_comparison.loc[df_comparison["Society"] != df_comparison["SOURCE"], "Difference"] = "Society&SOURCE"
df_comparison.loc[df_comparison["Source"] != df_comparison["INCOME-TYPE"], "Difference"] = "Source&INCOME-TYPE"
df_comparison.loc[df_comparison["Duration"] != df_comparison["DURATION"], "Difference"] = "Duration&DURATION"
df_comparison.loc[df_comparison["Amount"] != df_comparison["EARNINGS-AMOUNT"], "Difference"] = "Amount&EARNINGS"


# df = pd.concat([df_comparison[df_comparison["Use"] != df_comparison["USAGE"]], 
#                 df_comparison[df_comparison["Source"] != df_comparison["INCOME-TYPE"]],
#                 df_comparison[df_comparison["Duration"] != df_comparison["DURATION"]]])

In [39]:
# Visualization options
pd.set_option("display.max_columns", len(df_comparison))

In [52]:
df_comparison[df_comparison["Difference"] == "Society&SOURCE"]   # 342096 Entries

,Dist.,Distribution Date,Dist. Period Start,Dist. Period End,Member Name,Member Number,Member IPI No,Assignee Name,Assignee IPI No,Assignor Name,Assignor IPI No,AV Title,Episode,Work Title,Composer(s)/Author(s),Work Number,ISWC,Share %,Use,# of Plays,Society,Source,Station,Duration,Concert Date,Performer,Venue,City,Amount,DIST.,DIST-DATE,DIST-PERIOD-START,DIST-PERIOD-END,SPECIAL-DIST,SUPPLIER-NAME,SUPPLIER-NO,IPI NAME NO,SERIES-NO,SERIES-TITLE,AV-WORK-NO,AV-WORK-TITLE,SONG-TITLE,SONG-WRITER,SONG-CODE,ISWC-DAT,SHARE-PERCENT,USAGE,PERF-COUNT,SOCIETY,INCOME-TYPE,SOURCE,DURATION,PERF-DATE,PERFORMERS,VENUE,CITY,PERF-TYPE,EARNINGS-AMOUNT,Difference
0,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,,,BEDTRACKS_23311_BROTHERS,KIWENZIE BRYDEN GWISS/,202369442,T3026190041,50.00,,14.0,SOCAN,RCBC,,,,,,,40.5956,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,,,,BEDTRACKS_23311_BROTHERS,KIWENZIE BRYDEN GWISS/,202369442,T3026190041,50.00,,,,RCBC,,,,,,,,40.5956,Society&SOURCE
1,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,,,BEDTRACKS 21683 GROWTH,PRESTON KYLE D,90398592,T9238472327,50.00,F,102.0,SOCAN,DINA,ISPOTIFY,,,,,,0.0280,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,,,,BEDTRACKS 21683 GROWTH,PRESTON KYLE D,90398592,T9238472327,50.00,F,,,DINA,ISPOTIFY,,,,,,,0.0280,Society&SOURCE
2,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,,,BEDTRACKS 18242 LETITGO,ARINOS RINAT/HURWITZ S,87595464,T9248055881,16.67,F,769.0,SOCAN,YTUB,IYTUB,,,,,,0.1033,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,,,,BEDTRACKS 18242 LETITGO,ARINOS RINAT/HURWITZ S,87595464,T9248055881,16.67,F,,,YTUB,IYTUB,,,,,,,0.1033,Society&SOURCE
3,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,,,BEDTRACKS 20060 FEELTHE JU,ABRAHAM TIMOTHY PAUL,93472040,T9267713622,50.00,F,155.0,SOCAN,YTUB,IYTUB,,,,,,0.0624,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,,,,BEDTRACKS 20060 FEELTHE JU,ABRAHAM TIMOTHY PAUL,93472040,T9267713622,50.00,F,,,YTUB,IYTUB,,,,,,,0.0624,Society&SOURCE
4,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,,,BEDTRACKS_22397_RIVERGRATE,NATALE LOUIS FREDERICK,202380606,T3025084479,50.00,F,120.0,SOCAN,YTUB,IYTUB,,,,,,0.0483,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,,,,BEDTRACKS_22397_RIVERGRATE,NATALE LOUIS FREDERICK,202380606,T3025084479,50.00,F,,,YTUB,IYTUB,,,,,,,0.0483,Society&SOURCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382435,2Q12,2013-02-15,2012-04-01,2012-06-30,BEDTRACKS INC,7287577,669031729,,,,,16X9,EPISODE 134,16X9,ABRAHAM TI/JOHNSON AL,75024437,,50.00,B,27.0,SOCAN,TV,GBL,00:07:50,,,,,594.1500,2Q12,20130215.0,2012-04-01,2012-06-30,N,BEDTRACKS INC,7287577,69031729 2,A5221096,16X9,A5221098,EPISODE 134,16X9,ABRAHAM TI/JOHNSON AL,75024437,,50.00,B,27.0,,TV,GBL,00:07:50,,,,,,594.1500,Society&SOURCE
382436,2Q12,2013-02-15,2012-04-01,2012-06-30,BEDTRACKS INC,7287577,669031729,,,,,16X9,EPISODE 134,16X9,ABRAHAM TI/JOHNSON AL,75024437,,50.00,B,1.0,SOCAN,TV,LOCTV,00:07:50,,,,,6.6500,2Q12,20130215.0,2012-04-01,2012-06-30,N,BEDTRACKS INC,7287577,69031729 2,A5221096,16X9,A5221098,EPISODE 134,16X9,ABRAHAM TI/JOHNSON AL,75024437,,50.00,B,1.0,,TV,LOCTV,00:07:50,,,,,,6.6500,Society&SOURCE
382437,1Q12,2012-11-15,2012-01-01,2012-03-31,BEDTRACKS INC,7287577,669031729,,,,,SUZANNE ROGERS PRESENTS MARCHESA RED CARPET TO R,,SUZANNE ROGERS PRESENTS MA,ABRAHAM TI/JOHNSON AL,74217359,,50.00,B,1.0,SOCAN,TV,LOCTV,00:13:39,,,,,5.9500,1Q12,20121115.0,2012-01-01,2012-03-31,N,BEDTRACKS INC,7287577,69031729 2,A5024012,SUZANNE ROGERS PRESENTS MARCHESA RED CARPET TO R,A5024012,,SUZANNE ROGERS PRESENTS MA,ABRAHAM TI/JOHNSON AL,74217359,,50.00,B,1.0,,TV,LOCTV,00:13:39,,,,,,5.9500,Society&SOU

In [53]:
df_comparison[df_comparison["Difference"] == "Source&INCOME-TYPE"]   # 40322 Entries

,Dist.,Distribution Date,Dist. Period Start,Dist. Period End,Member Name,Member Number,Member IPI No,Assignee Name,Assignee IPI No,Assignor Name,Assignor IPI No,AV Title,Episode,Work Title,Composer(s)/Author(s),Work Number,ISWC,Share %,Use,# of Plays,Society,Source,Station,Duration,Concert Date,Performer,Venue,City,Amount,DIST.,DIST-DATE,DIST-PERIOD-START,DIST-PERIOD-END,SPECIAL-DIST,SUPPLIER-NAME,SUPPLIER-NO,IPI NAME NO,SERIES-NO,SERIES-TITLE,AV-WORK-NO,AV-WORK-TITLE,SONG-TITLE,SONG-WRITER,SONG-CODE,ISWC-DAT,SHARE-PERCENT,USAGE,PERF-COUNT,SOCIETY,INCOME-TYPE,SOURCE,DURATION,PERF-DATE,PERFORMERS,VENUE,CITY,PERF-TYPE,EARNINGS-AMOUNT,Difference
25578,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,A USER S GUIDE TO CHEATING DEATH,GERMS,USER S GUIDE TO CHEATING D,MCALLISTER ROB DANIEL,91180464,,50.0,,1.0,SGAE,TV,,,,,,,0.0138,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,A USER S GUIDE TO CHEATING DEATH,,GERMS,USER S GUIDE TO CHEATING D,MCALLISTER ROB DANIEL,91180464,,50.0,,,SGAE,FI,SGAE,,,,,,TV,0.0138,Source&INCOME-TYPE
25579,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,A USER S GUIDE TO CHEATING DEATH,SEX AND RELATIONSHIPS,USER S GUIDE TO CHEATING D,MCALLISTER ROB DANIEL,91180464,,50.0,,1.0,SGAE,TV,,,,,,,0.0138,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,A USER S GUIDE TO CHEATING DEATH,,SEX AND RELATIONSHIPS,USER S GUIDE TO CHEATING D,MCALLISTER ROB DANIEL,91180464,,50.0,,,SGAE,FI,SGAE,,,,,,TV,0.0138,Source&INCOME-TYPE
25580,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,A USER S GUIDE TO CHEATING DEATH,SPIRITUAL SCIENCE,BEDTRACKS 16913 COLDJOURNE,POPE ASHER ALEXANDER,85823568,T9248035714,50.0,,1.0,SGAE,TV,,,,,,,0.0138,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,A USER S GUIDE TO CHEATING DEATH,,SPIRITUAL SCIENCE,BEDTRACKS 16913 COLDJOURNE,POPE ASHER ALEXANDER,85823568,T9248035714,50.0,,,SGAE,FI,SGAE,,,,,,TV,0.0138,Source&INCOME-TYPE
25581,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,A USER S GUIDE TO CHEATING DEATH,SPIRITUAL SCIENCE,BEDTRACKS 20876 PICA,BLOEMERS HENDRIK HENK,90396414,T9248259112,50.0,,1.0,SGAE,TV,,,,,,,0.0138,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,A USER S GUIDE TO CHEATING DEATH,,SPIRITUAL SCIENCE,BEDTRACKS 20876 PICA,BLOEMERS HENDRIK HENK,90396414,T9248259112,50.0,,,SGAE,FI,SGAE,,,,,,TV,0.0138,Source&INCOME-TYPE
25582,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,A USER S GUIDE TO CHEATING DEATH,SPIRITUAL SCIENCE,USER S GUIDE TO CHEATING D,KHASKIN ALEXANDER,91180468,,50.0,,1.0,SGAE,TV,,,,,,,0.0138,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,,A USER S GUIDE TO CHEATING DEATH,,SPIRITUAL SCIENCE,USER S GUIDE TO CHEATING D,KHASKIN ALEXANDER,91180468,,50.0,,,SGAE,FI,SGAE,,,,,,TV,0.0138,Source&INCOME-TYPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378821,4Q14,2015-08-14,2014-10-01,2014-12-31,BEDTRACKS INC,7287577,669031729,,,,,TREASURES DECODED,SPEAR OF DESTINY,CORRIDOR OF SUSPENSE,JOHNSON OLIVER,80735834,T9158499333,25.0,,0.0,PRS,TV,,,,,,,0.9700,4Q14,20150814.0,2014-10-01,2014-12-31,N,BEDTRACKS INC,7287577,69031729 2,,TREASURES DECODED,,SPEAR OF DESTINY,CORRIDOR OF SUSPENSE,JOHNSON OLIVER,80735834,T9158499333,25.0,,,PRS,FI,PRS,,,,,,TV,0.9700,Source&INCOME-TYPE
378822,4Q14,2015-08-14,2014-10-01,2014-12-31,BEDTRACKS INC,7287577,669031729,,,,,TREASURES DECODED,SPEAR OF DESTINY,WAKING UP TO REALITY,OYE ANDREW PATRICK,80718925,T9158498498,50.0,,0.0,PRS,TV,,,,,,,1.4100,4Q14,20150814.0,2014-10-01,2014-12-31,N,BEDTRACKS INC,7287577,69031729 2,,TREASURES DECODED,,SPEAR OF DESTINY,WAKING UP TO REALITY,OYE ANDREW PATRICK,80718925,T9158498498,50.0,,,PRS,FI,PRS,,,,,,TV,1.4100,Source&INCOME-TYPE
379989,3Q14

In [54]:
df_comparison[df_comparison["Difference"] == "Duration&DURATION"]   # 22 entries

,Dist.,Distribution Date,Dist. Period Start,Dist. Period End,Member Name,Member Number,Member IPI No,Assignee Name,Assignee IPI No,Assignor Name,Assignor IPI No,AV Title,Episode,Work Title,Composer(s)/Author(s),Work Number,ISWC,Share %,Use,# of Plays,Society,Source,Station,Duration,Concert Date,Performer,Venue,City,Amount,DIST.,DIST-DATE,DIST-PERIOD-START,DIST-PERIOD-END,SPECIAL-DIST,SUPPLIER-NAME,SUPPLIER-NO,IPI NAME NO,SERIES-NO,SERIES-TITLE,AV-WORK-NO,AV-WORK-TITLE,SONG-TITLE,SONG-WRITER,SONG-CODE,ISWC-DAT,SHARE-PERCENT,USAGE,PERF-COUNT,SOCIETY,INCOME-TYPE,SOURCE,DURATION,PERF-DATE,PERFORMERS,VENUE,CITY,PERF-TYPE,EARNINGS-AMOUNT,Difference
8418,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,SOCAN,TV,CHEX,05:01:00,,,,,27.8202,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,A6557716,GLOBAL NEWS SPECIAL,A21967938,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,,TV,CHEX,03:01:00,,,,,,27.8202,Duration&DURATION
8420,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,SOCAN,TV,CHEX2,05:01:00,,,,,7.1334,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,A6557716,GLOBAL NEWS SPECIAL,A21967938,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,,TV,CHEX2,03:01:00,,,,,,7.1334,Duration&DURATION
8422,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,SOCAN,TV,CHFD,05:01:00,,,,,20.6868,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,A6557716,GLOBAL NEWS SPECIAL,A21967938,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,,TV,CHFD,03:01:00,,,,,,20.6868,Duration&DURATION
8424,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,SOCAN,TV,CIII,05:01:00,,,,,2100.0650,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,A6557716,GLOBAL NEWS SPECIAL,A21967938,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,,TV,CIII,03:01:00,,,,,,2100.0650,Duration&DURATION
8426,2Q22,2023-02-15,2022-04-01,2022-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,SOCAN,TV,CKWS,05:01:00,,,,,24.9668,2Q22,,2020-04-01,2020-06-30,N,BEDTRACKS INC,7287577,69031729,A6557716,GLOBAL NEWS SPECIAL,A21967938,2022 DECISION ONTARIO,DECISION CANADA,MCALLISTER ROB DANIEL,86882115,,50.0,B,1.0,,TV,CKWS,03:01:00,,,,,,24.9668,Duration&DURATION
101890,2Q21,2022-02-15,2021-04-01,2021-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2019 DECISION CANADA ADJUSTMENT,DECISION CANADA,MCALLISTER,86882115,,50.0,B,2.0,SOCAN,CA,BC1,04:17:00,,,,,24.7736,2Q21,20220215.0,2021-04-01,2021-06-30,N,BEDTRACKS INC,7287577,69031729 2,A6557716,GLOBAL NEWS SPECIAL,A21509816,2019 DECISION CANADA ADJUSTMENT,DECISION CANADA,MCALLISTER,86882115,,50.0,B,2.0,,CA,BC1,02:57:00,,,,,,24.7736,Duration&DURATION
101891,2Q21,2022-02-15,2021-04-01,2021-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2019 DECISION CANADA ADJUSTMENT,DECISION CANADA,MCALLISTER,86882115,,50.0,B,1.0,SOCAN,TV,CFRE,04:17:00,,,,,24.2717,2Q21,20220215.0,2021-04-01,2021-06-30,N,BEDTRACKS INC,7287577,69031729 2,A6557716,GLOBAL NEWS SPECIAL,A21509816,2019 DECISION CANADA ADJUSTMENT,DECISION CANADA,MCALLISTER,86882115,,50.0,B,1.0,,TV,CFRE,02:57:00,,,,,,24.2717,Duration&DURATION
101892,2Q21,2022-02-15,2021-04-01,2021-06-30,BEDTRACKS INC,7287577,669031729,,,,,GLOBAL NEWS SPECIAL,2019 DECISION CANADA ADJUSTMENT,DECISION CANADA,MCALLISTER,86882115,,50.0,B,1.0,SOCAN,TV,CFSK,04:17:00,,,,,

In [55]:
pd.reset_option("display.max_columns")